In [1]:
import os
import re
import glob
import pandas as pd
import numpy as np

In [2]:
import seaborn as sns

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
author = ["Chang_et_al", "Tse_et_al"][0]
task = ["Naming", "LD"][0]

data_folder = os.path.join("Data_Linguistic", author, task)

zscored = ["", "zscored_"][1]
data_paths = glob.glob(os.path.join(data_folder, f"{zscored}sub_*.xlsx"))

print(data_folder)
print("# Total:", len(data_paths), "files")

Data_Linguistic\Chang_et_al\Naming
# Total: 140 files


In [9]:
# nT_list = []
# for fp in data_paths:
#     data = pd.read_excel(fp)
#     nT_list.append(len(data))
    
# print(
#     np.array(nT_list).mean(), 
#     np.array(nT_list).min(), 
#     np.array(nT_list).max(), 
#     np.array(nT_list).std()
# )

160.92857142857142 131 198 13.07158469852115


In [5]:
out_folder = os.path.join(data_folder, "derinatives")

if not os.path.exists(out_folder):
    os.mkdir(out_folder)
    
print(out_folder)

Data_Linguistic\Chang_et_al\Naming\derinatives


In [6]:
sidx = 1

fp = data_paths[sidx]
fn = os.path.basename(fp)
sid = re.findall("sub_([\d]{3}).xlsx", fn)[0]

print("# subj ID: ", sid)

# subj ID:  102


In [7]:
data = pd.read_excel(fp)

In [8]:
data.columns

Index(['LogCF', 'NS', 'CON', 'PC', 'SC', 'SAR', 'IMG', 'AoA', 'z_rt'], dtype='object')

In [11]:
if zscored == "zscored_": 
    targ_col = "z_rt"
    fn = f"[histplot] {targ_col} (z-scored) sub-{sid}.png"
    bin_range = (-3, 3)
else:
    targ_col = "rt"
    fn = f"[histplot] {targ_col} sub-{sid}.png"
    bin_range = None
    
fd = os.path.join("Figs_Linguistic", author, task)
if not os.path.exists(fd):
    os.makedirs(fd) 

fig, ax = plt.subplots(figsize=(8, 6))
sns.histplot(
    data=data, 
    x=targ_col, 
    binrange=bin_range, 
    kde=True, 
    bins=30
)
ax.set_xlabel(targ_col, fontsize=16)
ax.set_ylabel("Count", fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout() 
plt.savefig(os.path.join(fd, fn), format='png', dpi=200)
plt.close()

print(f"saved: {fn}")

saved: [histplot] z_rt (z-scored) sub-102.png


In [ ]:
# plt.style.use('seaborn-v0_8-white')

# ax.hist(
#     data[targ_col], 
#     density=True, 
#     bins=30, 
#     # range=[-3, 3], 
#     alpha=0.5
# )

In [127]:
np.quantile(data[F1], [.25, .5, .75])

array([2.03541625, 2.74351   , 3.3854685 ])

In [118]:
F1 = "LogCF"
F2 = "IMG"
Y  = "rt"
nF = 2
min_nT = (nF*3 + nF^2 + 2) / 2

sub_data = data.loc[:, [F1, F2, Y]]
keep_going = True
count = 0

while keep_going:
    count = count + 1
    
    sub_data["F1_Q"] = pd.qcut(sub_data[F1], 4, labels=["Q1", "Q2", "Q3", "Q4"])
    sub_data["F2_Q"] = pd.qcut(sub_data[F2], 4, labels=["Q1", "Q2", "Q3", "Q4"])  
    
    out_data = (sub_data
                .query('F1_Q in ["Q1", "Q4"] & F2_Q in ["Q1", "Q4"]')
                .iloc[:, :3])
    
    if len(out_data) > min_nT:
        fn = f"sub-{sid}_part-{count}.xlsx"
#         out_data.to_excel(os.path.join(out_folder, fn), index=False)
    else:
        keep_going = False
        
    sub_data = sub_data.drop(out_data.index)
    
    print(len(out_data), len(sub_data))

62 164
32 132
22 110
18 92
13 79
11 68
8 60
3 57
